In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:
GAN_MODEL_PATH='~/pl0025.ckpt'
CLASS_MODEL_PATH='../logs/runs/2022-04-26/15-02-42/checkpoints/last.ckpt'
CLASS_MODEL_PATH2='../logs/runs/2022-04-09/20-13-27/checkpoints/last.ckpt'
IMAGE_FOLDER = '../data/unlid'
SEED = 12345

In [3]:
import pytorch_lightning as pl
pl.seed_everything(SEED, workers=True)

Global seed set to 12345


12345

In [4]:
from src.models.resnet_model import ResnetModel
resnet1 = ResnetModel.load_from_checkpoint(CLASS_MODEL_PATH)
resnet1 = resnet1.model

In [5]:
resnet2 = ResnetModel.load_from_checkpoint(CLASS_MODEL_PATH2)
resnet2 = resnet2.model

In [6]:
from src.datamodules.ec_datamodule import EnhancedClassifierDatamodule

ec_dm = EnhancedClassifierDatamodule('../data/', GAN_MODEL_PATH,dataset_name= "unlid/A", dataset_name_hallucination= "unlid/B", hallucination_percentage=0,
train_val_test_split=(0.7, 0.15, 0.15), pin_memory=True, batch_size=124, num_workers=5 )

ec_dm.setup()

len(ec_dm.data_test)

C:\ProgramData\Miniconda3\envs\herbarium-time-machine\lib\site-packages\torchmetrics\utilities\prints.py:37: UserWarning: Metric `FID` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\ProgramData\Miniconda3\envs\herbarium-time-machine\lib\site-packages\torchmetrics\utilities\prints.py:37: UserWarning: Metric `IS` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


1442

In [7]:
import torch
from torchmetrics import Accuracy

resnet1 = resnet1.cuda()
resnet1.eval()

resnet2 = resnet2.cuda()
resnet2.eval()

with torch.no_grad():
    acc1 = Accuracy(num_classes=255, average='none').cuda()
    acc2 = Accuracy(num_classes=255, average='none').cuda()
    test_loader = ec_dm.test_dataloader()

    for data, target in test_loader:
        images = data.cuda()
        labels = target.cuda()
        outputs1 = resnet1(images)
        outputs2 = resnet2(images)

        acc1(outputs1, labels)
        acc2(outputs2, labels)
    
    pacc1 = acc1.compute()
    pacc2 = acc2.compute()

    # print('Test Accuracy of the model:',pacc)

C:\ProgramData\Miniconda3\envs\herbarium-time-machine\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [8]:
import pandas as pd
pacc1c = pacc1.clone()
pacc2c = pacc2.clone()
pacc1c.cpu()
pacc2c.cpu()
results=pd.DataFrame(columns=['Species', 'AccuracyHallucination', 'AccuracyNormal'])
for class_ in ec_dm.data_test.dataset.classes:
    idx = ec_dm.data_test.dataset.class_to_idx[class_]
    results = results.append({'Species':class_, 'AccuracyHallucination':pacc1c[idx].item(), 'AccuracyNormal':pacc2c[idx].item()}, ignore_index=True)
    # results.loc[class_] = pacc2[idx].item()
print(results)
results.to_csv('test.csv')

                     Species  AccuracyHallucination  AccuracyNormal
0       Acnistus arborescens               0.888889             1.0
1         Aegiphila valerioi               1.000000             1.0
2    Allophylus psilospermus               1.000000             1.0
3      Alvaradoa amorphoides               1.000000             1.0
4         Ambrosia peruviana               1.000000             1.0
..                       ...                    ...             ...
250     Trichilia havanensis               0.750000             1.0
251           Ulmus mexicana               1.000000             1.0
252         Urera caracasana               1.000000             1.0
253     Verbesina ovatifolia               1.000000             1.0
254          Vernonia patens               1.000000             1.0

[255 rows x 3 columns]
